# Importing Data

In [1]:
from msg_parsing import OrderBook
import pandas as pd
x = OrderBook('3738 HK Equity')
x.run()

100%|██████████| 1/1 [00:00<00:00,  9.20it/s]


In [3]:
x.all_msg_df

,MKTDEPTH_EVENT_TYPE,MKTDEPTH_EVENT_SUBTYPE,ID_BB_SEC_NUM_SRC,FEED_SOURCE,EID,MD_TABLE_CMD_RT,MD_BOOK_TYPE,MBL_WINDOW_SIZE,MD_BOOK_SUBTYPE,MD_PRICE_DISPLAY_FORMAT_TYPE_RT,...,MBL_ASK_RT,MBL_ASK_COND_CODE_RT,MBL_ASK_NUM_ORDERS_RT,MBL_ASK_SIZE_RT,MBO_ASK_POSITION_RT,MBO_ASK_RT,MBO_ASK_BROKER_RT,MBO_ASK_COND_CODE_RT,MBO_ASK_SIZE_RT,MBO_ORDER_ID_RT
0,MARKET_BY_LEVEL,TABLE_INITPAINT,7.302263e+11,HK,14112,REPLACE,MBL-RBP,10.0,CLOB,DECIMAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MARKET_BY_ORDER,TABLE_INITPAINT,7.302263e+11,HK,53714,ADD,MBO-AMD,NaN,CLOB,DECIMAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MARKET_BY_ORDER,TABLE_INITPAINT,7.302263e+11,HK,14112,REPLACE,TOP-RBP,NaN,TOPBROKER,DECIMAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MARKET_BY_LEVEL,BID,NaN,NaN,14112,REPLACE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MARKET_BY_ORDER,BID,NaN,NaN,14112,REPLACE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,MARKET_BY_ORDER,BID,NaN,NaN,14112,REPLACE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,MARKET_BY_ORDER,BID,NaN,NaN,14112,REPLACE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,MARKET_BY_ORDER,BID,NaN,NaN,14112,REPLACE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,MARKET_BY_ORDER,BID,NaN,NaN,14112,REPLACE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
x.trade_prints

,time,type,value,size,conditionCodes
0,2023-06-23 09:10:00.030,TRADE,2.6,4000,IE
1,2023-06-23 09:20:28.772,TRADE,2.6,4000,U


In [11]:
pd.to_pickle(x.all_msg_df, 'res/trade_recordings/3738_HK_all_msg_df.pickle')
pd.to_pickle(x.trade_prints, 'res/trade_recordings/3738_HK_trade_prints.pickle')

# Matching Auction Orders

In [12]:
import pandas as pd
all_msg_df = pd.read_pickle('res/trade_recordings/3738_HK_all_msg_df.pickle')
trade_prints = pd.read_pickle('res/trade_recordings/3738_HK_trade_prints.pickle')

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pymongo import MongoClient

ticker = '3738 HK Equity'
date = '20230705'

# MongoDB connection
client = MongoClient('mongodb://192.168.10.153:27017/')
db = client["BPIPE"][date]

# Retrieve messages
cursor = db.find({"cid": {"$regex": ticker}})
all_msg_df = pd.DataFrame(list(cursor)).drop(["_id"], axis=1).dropna(subset=["MKTDEPTH_EVENT_TYPE"])

# all_msg_df = pd.read_pickle('Z:/External Research/2023 Interns/BPIPE/res/trade_recordings/3738_HK_all_msg_df.pickle')
# trade_prints = pd.read_pickle('Z:/External Research/2023 Interns/BPIPE/res/trade_recordings/3738_HK_trade_prints.pickle')
            
def update_mbl(event, mbl_bid, mbl_ask, df_bid, df_ask):
    """Update the MBL orderbook before auction"""
    side = event['MKTDEPTH_EVENT_SUBTYPE'].iloc[0]
    mbl_new = mbl_bid.copy() if side == 'BID' else mbl_ask.copy()
    mbl_old = mbl_bid.copy() if side == 'BID' else mbl_ask.copy()
    df = df_bid.copy() if side == 'BID' else df_ask.copy()

    # Update the MBL
    for _, row in event.iterrows(): 
        if row['MD_TABLE_CMD_RT'] == 'REPLACE':
            mbl_new.loc[row[f'MBL_{side}_POSITION_RT']] = [row['MBL_TIME_RT'], row[f'MBL_{side}_POSITION_RT'], row[f'MBL_{side}_RT'], row[f'MBL_{side}_NUM_ORDERS_RT'], row[f'MBL_{side}_SIZE_RT']] # Replace the position in current top orderbook
        elif row['MD_TABLE_CMD_RT'] == 'REPLACE_CLEAR':
            mbl_new.drop(row[f'MBL_{side}_POSITION_RT'], inplace=True) # Clear the position in the top order book 
    # TODO: when a new price level is added, there could be multiple orders added at the same time; the same for adding orders on existing price level.
    # Compare existing MBL and updated MBL to get an order book in bid side
    if len(mbl_old) != len(mbl_new): # A price level is added or removed in the MBL bid
        mbl_position = event[f'MBL_{side}_POSITION_RT'].iloc[0] 
        order_position = mbl_new.loc[:(mbl_position-1)]['Orders'].sum() + 1 if mbl_position != 1 else 1.0
        if len(mbl_old) < len(mbl_new): # A price level is added
            df['Position'] = df['Position'].apply(lambda x: x + 1 if x >= order_position else x) # Inferior positions shift down
            df.loc[len(df)] = [event['MBL_TIME_RT'].iloc[0], order_position, event[f'MBL_{side}_RT'].iloc[0], event[f'MBL_{side}_SIZE_RT'].iloc[0]] # Add the new position
            df.sort_values(by='Position', inplace=True, ignore_index=True)
        elif len(mbl_old) > len(mbl_new): # A price level is removed
            del_size = mbl_old.loc[mbl_position, 'Orders']
            drop_index = df[df['Position'].isin(range(int(order_position), int(order_position + del_size)))].index
            df.drop(drop_index, inplace=True) # Remove the existing position
            df['Position'] = df['Position'].apply(lambda x: x - del_size if x >= order_position + del_size else x) # Inferior positions shift up 
            df.sort_values(by='Position', inplace=True, ignore_index=True)
    elif len(mbl_old) == len(mbl_new): # An order is added, removed or modified in existing price levels
        diff = mbl_new.compare(mbl_old)
        mbl_position = diff.index[0] # Find which price level has order change
        order_position = mbl_new.loc[:mbl_position]['Orders'].sum()
        order_diff = diff.loc[mbl_position, ('Orders', 'self')] - diff.loc[mbl_position, ('Orders', 'other')] # Difference of number of orders: + 1 means adding new order; -1 means remove one order
        size_diff = diff.loc[mbl_position, ('Size', 'self')] - diff.loc[mbl_position, ('Size', 'other')] # Difference of size
        if order_diff == 1: # Add new order
            df['Position'] = df['Position'].apply(lambda x: x + 1 if x >= order_position else x) # Inferior positions shift down by 1
            df.loc[len(df)] = [event['MBL_TIME_RT'].iloc[0], order_position, event[f'MBL_{side}_RT'].iloc[0], size_diff] # Add the new position
            df.sort_values(by='Position', inplace=True, ignore_index=True)
        elif order_diff == -1: # Remove one order
            bid_range_lower = mbl_new.loc[:mbl_position]['Orders'].sum() 
            bid_range_upper = mbl_new.loc[:(mbl_position + 1)]['Orders'].sum()
            bid_range = df.loc[bid_range_lower:bid_range_upper]
            order_position = bid_range[bid_range['Size']  == -size_diff]['Position'].iloc[0] 
            df.drop(df[df['Position'] == order_position].index, inplace=True) # Remove the existing position
            df['Position'] = df['Position'].apply(lambda x: x - 1 if x > order_position else x) # Inferior positions shift up by 1
            df.sort_values(by='Position', inplace=True, ignore_index=True)
    
    if side == 'BID':
        mbl_bid = mbl_new
        df_bid = df
    elif side == 'ASK':
        mbl_ask = mbl_new
        df_ask = df
    return mbl_bid, mbl_ask, df_bid, df_ask

def match_auction(all_msg_df: pd.DataFrame, trade_prints: pd.DataFrame):
    """Match auction trades"""
    end_time = trade_prints[trade_prints['conditionCodes']=='U']['time'].iloc[0] # Find the time starting auction match
    mbl_df = all_msg_df[(all_msg_df['MKTDEPTH_EVENT_TYPE']=='MARKET_BY_LEVEL') & (all_msg_df['EID']==14112) & (all_msg_df['MKTDEPTH_EVENT_SUBTYPE']!="TABLE_INITPAINT") & (all_msg_df['MBL_TIME_RT'] < end_time - timedelta(hours=8))] 
    top_df = all_msg_df[(all_msg_df['MKTDEPTH_EVENT_TYPE']=='MARKET_BY_ORDER') & (all_msg_df['EID']==14112) & (all_msg_df['MKTDEPTH_EVENT_SUBTYPE']!="TABLE_INITPAINT") & (all_msg_df['MBO_TIME_RT'] < end_time - timedelta(hours=8))] 
    trade_df = trade_prints[trade_prints['conditionCodes']=='U']

    # Initialize dataframe
    top_bid = pd.DataFrame(columns=['Timestamp', 'Position', 'Price', 'BrokerID'])
    top_ask = pd.DataFrame(columns=['Timestamp', 'Position', 'Price', 'BrokerID'])
    mbl_bid = pd.DataFrame(columns=['Timestamp', 'Position', 'Price', 'Orders', 'Size'])
    mbl_ask = pd.DataFrame(columns=['Timestamp', 'Position', 'Price', 'Orders', 'Size'])
    df_bid = pd.DataFrame(columns=['Timestamp', 'Position', 'Price', 'Size'])
    df_ask = pd.DataFrame(columns=['Timestamp', 'Position', 'Price', 'Size'])

    # Update TOP till auction begins
    for ix, event in top_df.iterrows():
        side = event["MKTDEPTH_EVENT_SUBTYPE"]
        df = top_bid if side == 'BID' else top_ask
        if event["MD_TABLE_CMD_RT"] == "REPLACE":
            df.loc[event[f"MBO_{side}_POSITION_RT"]] = [event["MBO_TIME_RT"], event[f"MBO_{side}_POSITION_RT"], event[f"MBO_{side}_RT"], event[f"MBO_{side}_BROKER_RT"]]
        elif event["MD_TABLE_CMD_RT"] == "REPLACE_CLEAR":
            df.drop(event[f"MBO_{side}_POSITION_RT"], inplace=True)
        if side == 'BID':
            top_bid = df
        elif side == "ASK":
            top_ask = df

    # Update MBL and order book till auction begins
    mbl_df['group_indicator'] = mbl_df['MD_MULTI_TICK_UPD_RT'].map({0: 0, 1: np.nan})
    mbl_df['group_indicator'].loc[mbl_df['group_indicator']==0] = range(mbl_df['group_indicator'].value_counts()[0])
    mbl_df['group_indicator'] = mbl_df['group_indicator'].fillna(method='bfill') # Create groupby key

    for ix, group in mbl_df.groupby(['group_indicator']):
        mbl_bid, mbl_ask, df_bid, df_ask = update_mbl(group, mbl_bid=mbl_bid, mbl_ask=mbl_ask, df_bid=df_bid, df_ask=df_ask)
    
    # Map brokers to the order book
    df_bid = df_bid.merge(top_bid.drop('Timestamp', axis=1), how='left', on=['Position', 'Price'])
    df_ask = df_ask.merge(top_ask.drop('Timestamp', axis=1), how='left', on=['Position', 'Price'])

    # Match trades
    auction_journal = [] # "Timestamp", "BrokerID", "Price", "Side", "Size", "Action", "Position", 'Condition Codes'
    for ix, trade in trade_df.iterrows():
        trade_price = trade['value']
        trade_size = trade['size']
        auction_journal.append([trade['time'], df_bid['BrokerID'].iloc[0], trade_price, 'BID', trade_size, 'Trade', 1, 'U'])   
        auction_journal.append([trade['time'], df_ask['BrokerID'].iloc[0], trade_price, 'ASK', trade_size, 'Trade', 1, 'U'])
        # Both first bid and first ask position is fully executed.
        if trade_price == df_bid['Price'].iloc[0] and trade_size == df_bid['Size'].iloc[0] and trade_price == df_ask['Price'].iloc[0] and trade_size == df_ask['Size'].iloc[0]:
            df_bid.drop(index=0, inplace=True, ignore_index=True)
            df_ask.drop(index=0, inplace=True, ignore_index=True)
        # First bid position is fully executed but ask position is partially executed.
        elif trade_price == df_bid['Price'].iloc[0] and trade_size == df_bid['Size'].iloc[0] and trade_price == df_ask['Price'].iloc[0] and trade_size != df_ask['Size'].iloc[0]:
            df_bid.drop(index=0, inplace=True, ignore_index=True)
            df_ask.loc[0, 'Size'] = df_ask.loc[0, 'Size'] - trade_size
        # First ask position is fully executed but bid position is partially executed.
        elif trade_price == df_ask['Price'].iloc[0] and trade_size != df_ask['Size'].iloc[0] and trade_price == df_ask['Price'].iloc[0] and trade_size != df_ask['Size'].iloc[0]: 
            df_ask.drop(index=0, inplace=True, ignore_index=True)
            df_bid.loc[0, 'Size'] = df_bid.loc[0, 'Size'] - trade_size
    
    auction_journal = pd.DataFrame(auction_journal, columns=["Timestamp", "BrokerID", "Price", "Side", "Size", "Action", "Position", 'Condition Codes'])
    
    return auction_journal

match_auction(all_msg_df, trade_prints)